In [2]:
from database_utils import DatabaseConnector, build_dataframe

db_connector = DatabaseConnector('localhost', 'root', '12345', 'CORPUS_VIES')
retrieved_data = build_dataframe(db_connector.getDataTextAndLabel())

retrieved_data

SELECT PARAGRAPH, POLARITY FROM PARAGRAPHS WHERE POLARITY IS NOT NULL AND trim(POLARITY) <> ""
1042  Paragraphs encountered


,labels,texts
0,NE,Os candidatos à Presidência mais bem posiciona...
1,NE,As maiores doadoras para a campanha de Dilma s...
2,NE,Os maiores colaboradores da campanha de Aécio ...
3,NE,A candidatura do PSB inicialmente com Eduardo...
4,NE,Em dois meses a candidatura de Dilma foi que ...
5,NG,O presidente do PT Rui Falcão acusou na noit...
6,NG,Aécio faz coro a denúncias sem provas veicula...
7,NG,Segundo Falcão é cômico ouvir alguém do PS...
8,NG,Ele citou o mensalão mineiro o esquema de cor...
9,NG,Para o presidente do PT Aécio deveria se preo...


# Balanceamento dos dados

In [1]:
from database_utils import DatabaseConnector, build_dataframe
from machine_learning_utils import MLWrapper

print("Recuperando os textos")
db_connector = DatabaseConnector('localhost', 'root', '12345', 'CORPUS_VIES')
retrieved_data = build_dataframe(db_connector.getDataTextAndLabel())

print("Treinando o modelo")
ml_wrapper = MLWrapper()
ml_wrapper.train(retrieved_data)

Recuperando os textos
SELECT PARAGRAPH, POLARITY FROM PARAGRAPHS WHERE POLARITY IS NOT NULL AND trim(POLARITY) <> ""
1042  Paragraphs encountered
Treinando o modelo


ValueError: operands could not be broadcast together with shapes (2,3) (3,3) (2,3) 